In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import pandas as pd
import sys
sys.path.append('.')  # Добавляем текущую директорию в путь поиска модулей
from utils import search_history
from utils import make_recommendations
from utils import get_recommended_homes


user_id=666
user_type=3

user_actions = search_history(user_id, user_type)
homes = pd.read_csv('data/homes_fixed_scaled.csv')
recs = make_recommendations(homes, user_actions)

ImportError: cannot import name 'get_recommended_homes' from 'utils' (/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/utils.py)

ImportError: cannot import name 'get_recommended_homes' from 'utils' (/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/utils.py)

In [9]:
recs[user_id]

[7749, 8669, 7705, 6609, 8828, 8182, 6845, 4773, 9344, 8419]

In [15]:
def get_recommended_homes(recommendation, user_id):
    homes_origin = pd.read_csv('data/homes_new.csv')
    
    # Фильтруем только рекомендованные дома
    recommended_ids = recommendation[user_id]
    filtered = homes_origin[homes_origin['id'].isin(recommended_ids)]
    
    # Сортируем по порядку в списке recommendation[user_id]
    filtered = filtered.set_index('id').loc[recommended_ids].reset_index()

    return filtered.drop(columns='geo_sum')



In [16]:
test = get_recommended_homes(recs, user_id)

In [17]:
test

,id,price,level,rooms,area,kitchen_area,geo_lat,geo_lon
0,7749,18890000,9,3,59.40,6.7,55.675778,37.511436
1,8669,18944664,9,3,60.18,0.0,55.754047,37.620405
2,7705,18999000,9,3,58.20,6.0,55.645395,37.522863
3,6609,18523120,7,2,60.80,0.0,55.808689,37.647212
4,8828,18500000,6,3,63.30,7.1,55.832480,37.568713
5,8182,18506500,7,3,63.43,0.0,55.782041,37.509694
6,6845,18540102,13,2,61.90,0.0,55.810503,37.701472
7,4773,18800000,6,2,60.00,9.6,55.798470,37.389420
8,9344,18549825,11,3,62.30,9.5,55.700243,37.919974
9,8419,19500000,4,3,64.10,0.0,55.721560,37.619719


In [18]:
final_recommendations = recs
homes_fixed = homes.copy()

In [19]:
# Создадим структуру для хранения результатов тестов
stat_test_results = {}

# Выполнение t-теста для каждого пользователя
for user in final_recommendations.keys():
    interacted_obj_ids = user_actions[user_actions['user_id'] == user]['obj_id']
    recommended_obj_ids = final_recommendations[user]

    interacted_objects = homes_fixed.loc[homes_fixed.index.intersection(interacted_obj_ids)]
    recommended_objects = homes_fixed.loc[homes_fixed.index.intersection(recommended_obj_ids)]

    if not interacted_objects.empty and not recommended_objects.empty:
        # Средние значения по объектам
        interacted_mean = interacted_objects.mean(axis=0)
        recommended_mean = recommended_objects.mean(axis=0)

        # Проведение t-теста
        t_stat, p_val = ttest_ind(interacted_mean, recommended_mean, equal_var=False, nan_policy='omit')

        # Сохранение результатов
        stat_test_results[user] = {'t_stat': t_stat, 'p_val': p_val}

        # Вывод результатов для каждого пользователя
        print(f'User {user}: t_stat = {t_stat:.5f}, p_val = {p_val:.5f}')

# Преобразуем в DataFrame для удобного анализа
stat_test_results_df = pd.DataFrame.from_dict(stat_test_results, orient='index')

# Подсчет доли пользователей, у которых p_val > 0.05
no_significant_diff_ratio = (stat_test_results_df['p_val'] > 0.05).mean()
print(f'Доля пользователей, у которых нет статистически значимой разницы: {no_significant_diff_ratio:.2%}')


User 666: t_stat = -0.00778, p_val = 0.99390
Доля пользователей, у которых нет статистически значимой разницы: 100.00%


если p_val > 0.05, то нет оснований полагать, что есть стат. значимая разница => рекомендации были хорошими для всех юзеров